In [22]:
import os
import requests
import pandas as pd
from bs4 import BeautifulSoup
import datetime

In [23]:
d = datetime.date.today()

In [24]:
d.strftime('%d %B, %Y')

'26 April, 2020'

In [28]:
def getCurrentWeather(params):
    weather = 'http://api.openweathermap.org/data/2.5/weather'
    params['APPID'] = '1714486ff1a0070e51d0757147b0eeb0'
    params['units'] = 'metric'
    return requests.get(weather,params)

In [120]:
params = {'q':'Los Angeles'}

In [121]:
response = getCurrentWeather(params)

In [122]:
response.status_code

200

In [123]:
response.json()

{'base': 'stations',
 'clouds': {'all': 1},
 'cod': 200,
 'coord': {'lat': 34.05, 'lon': -118.24},
 'dt': 1587915474,
 'id': 5368361,
 'main': {'feels_like': 19.65,
  'humidity': 64,
  'pressure': 1014,
  'temp': 19.4,
  'temp_max': 23.33,
  'temp_min': 14},
 'name': 'Los Angeles',
 'sys': {'country': 'US',
  'id': 3694,
  'sunrise': 1587906491,
  'sunset': 1587954769,
  'type': 1},
 'timezone': -25200,
 'visibility': 8047,
 'weather': [{'description': 'haze',
   'icon': '50d',
   'id': 721,
   'main': 'Haze'}],
 'wind': {'deg': 205, 'speed': 0.71}}

In [124]:
cit = response.json()

In [125]:
tim = cit['dt'] - 60*60*3 + cit['timezone']

In [127]:
datetime.datetime.fromtimestamp(tim).strftime("%Y-%m-%d %H:%M")

'2020-04-26 08:37'

## Getting World Cities

In [2]:
world_cities = requests.get('https://worldpopulationreview.com/world-cities/')

In [3]:
cities = BeautifulSoup(world_cities.text,'lxml')

In [4]:
city_names = []
for i in range(42,450):
    if i%2 == 0:
        city_names.append(cities.findAll('a')[i].getText())

In [5]:
city_names[:5]

['Tokyo', 'Delhi', 'Shanghai', 'Sao Paulo', 'Mexico City']

In [12]:
city_names_pop = pd.DataFrame(data={"city_names":city_names, "population":city_populations})

In [21]:
city_names_pop.to_csv('city_names.csv',index=False)

In [24]:
city_np = pd.read_csv('city_names.csv')

In [28]:
if city_np is None:
    print(3)

In [29]:
city_np['city_names'].values.tolist()

['Tokyo',
 'Delhi',
 'Shanghai',
 'Sao Paulo',
 'Mexico City',
 'Dhaka',
 'Cairo',
 'Beijing',
 'Mumbai',
 'Osaka',
 'Karachi',
 'Chongqing',
 'Istanbul',
 'Buenos Aires',
 'Kolkata',
 'Lagos',
 'Kinshasa',
 'Manila',
 'Tianjin',
 'Rio De Janeiro',
 'Guangzhou',
 'Lahore',
 'Moscow',
 'Shenzhen',
 'Bangalore',
 'Paris',
 'Bogota',
 'Chennai',
 'Jakarta',
 'Lima',
 'Bangkok',
 'Hyderabad',
 'Seoul',
 'Nagoya',
 'London',
 'Chengdu',
 'Tehran',
 'New York',
 'Wuhan',
 'Luanda',
 'Ahmedabad',
 'Xian',
 'Kuala Lumpur',
 'Hangzhou',
 'Hong Kong',
 'Dongguan',
 'Foshan',
 'Riyadh',
 'Shenyang',
 'Surat',
 'Baghdad',
 'Suzhou Jiangsu',
 'Santiago',
 'Dar Es Salaam',
 'Pune',
 'Madrid',
 'Harbin',
 'Toronto',
 'Belo Horizonte',
 'Singapore',
 'Khartoum',
 'Johannesburg',
 'Qingdao',
 'Dalian',
 'Barcelona',
 'Fukuoka',
 'St Petersburg',
 'Jinan',
 'Yangon',
 'Zhengzhou',
 'Alexandria',
 'Abidjan',
 'Guadalajara',
 'Ankara',
 'Chittagong',
 'Melbourne',
 'Sydney',
 'Monterrey',
 'Nairobi',
 'Br

In [15]:
for i in range(0,5):
    print(city_names_pop['city_names'].values.tolist()[i])

Tokyo
Delhi
Shanghai
Sao Paulo
Mexico City


In [12]:
len(city_names)

204

In [9]:
city_populations = []
for i in range(1220):
    if i%6 == 0:
        city_populations.append(cities.findAll('td')[i+3].getText())

In [10]:
city_populations[:5]

['37,393,129', '30,290,936', '27,058,479', '22,043,028', '21,782,378']

In [11]:
len(city_populations)

204

## Putting it all together

In [16]:
def get_correct_name(city_name):
    correct_name = city_name
    if city_name == 'St Petersburg':
        correct_name = 'St. Petersburg'
    elif city_name == 'Rome':
        correct_name = 'Rome,IT'
    elif city_name == 'Melbourne':
        correct_name = 'Melbourne,AU'
    return correct_name

In [20]:
world_temperatures = []
for i in range(204):
    params['q'] = get_correct_name(city_names[i])
    response = getCurrentWeather(params)
    if response.status_code != 200:
        continue
    city_information = response.json()
    country = city_information['sys']['country']
    latitude = city_information['coord']['lat']
    longitude = city_information['coord']['lon']
    weather_condition = city_information['weather'][0]['main']
    weather_det = city_information['weather'][0]['description']
    feels_like = city_information['main']['feels_like']
    temp = city_information['main']['temp']
    max_temp = city_information['main']['temp_max']
    min_temp = city_information['main']['temp_min']
    wind_speed = city_information['wind']['speed']
    city_data = {'City': city_names[i],
                 'Country': country,
                 'Date': d.strftime('%d-%m-%Y'),
                 'Latitude': latitude,
                 'Longitude': longitude,
                 'Population': city_populations[i],
                 'Weather': weather_condition,
                 'Main_Weather': weather_det,
                 'Feels_like': feels_like,
                 'temp': temp,
                 'max_temp': max_temp,
                 'min_temp': min_temp,
                 'Wind_Speed': wind_speed}
    world_temperatures.append(city_data)

world_temps_df = pd.DataFrame(world_temperatures, columns = city_data.keys())

In [28]:
world_temps_df.to_csv('weather.csv')